### Import to Neo4j

In [3]:
from neo4j import GraphDatabase
import csv
import os

# Neo4j connection parameters
URI = "bolt://localhost:7687"  
USERNAME = "neo4j"  
PASSWORD = "Password1234"  

class Neo4jImporter:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))
        
    def close(self):
        self.driver.close()
        
    def clear_database(self):
        with self.driver.session() as session:
            
            session.run("MATCH (n) DETACH DELETE n")
            print("Database cleared.")
    
    def create_constraints(self):
        with self.driver.session() as session:
            
            try:
                session.run("CREATE CONSTRAINT station_id_unique IF NOT EXISTS FOR (s:Station) REQUIRE s.id IS UNIQUE")
                print("Constraint created.")
            except Exception as e:
                
                print(f"Note: {e}")
                
    def import_stations(self, csv_file):
        station_count = 0
        with open(csv_file, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            with self.driver.session() as session:
                for row in reader:
                    
                    query = """
                    CREATE (s:Station {
                        id: $id,
                        name: $name,
                        x_coord: toFloat($x_coord),
                        y_coord: toFloat($y_coord),
                        type: $type
                    })
                    """
                    session.run(query, 
                               id=row['StationID'], 
                               name=row['Name'], 
                               x_coord=row['X_Coordinate'], 
                               y_coord=row['Y_Coordinate'], 
                               type=row['Type'])
                    station_count += 1
        print(f"Imported {station_count} stations.")
                
    def import_connections(self, csv_file):
        connection_count = 0
        with open(csv_file, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            with self.driver.session() as session:
                for row in reader:
                    
                    query = """
                    MATCH (a:Station {id: $source}), (b:Station {id: $target})
                    CREATE (a)-[r:CONNECTS_TO {
                        distance: toFloat($distance),
                        passengers: toInteger($passengers)
                    }]->(b)
                    """
                    session.run(query, 
                               source=row['SourceStationID'], 
                               target=row['TargetStationID'], 
                               distance=row['Distance_km'], 
                               passengers=row['DailyPassengers'])
                    connection_count += 1
        print(f"Imported {connection_count} connections.")


stations_file = os.path.join('data', 'stations.csv')
connections_file = os.path.join('data', 'connections.csv')


if __name__ == "__main__":
    try:
        
        importer = Neo4jImporter(URI, USERNAME, PASSWORD)
        print("Connected to Neo4j database.")
        
        
        importer.clear_database()
        importer.create_constraints()
        
        # Import data
        print("Importing station data...")
        importer.import_stations(stations_file)
        print("Importing connection data...")
        importer.import_connections(connections_file)
        
        print("Import complete!")
        importer.close()
    except Exception as e:
        print(f"Error: {e}")

Connected to Neo4j database.
Database cleared.
Constraint created.
Importing station data...
Imported 50 stations.
Importing connection data...
Imported 167 connections.
Import complete!
